In [ ]:
import pypsa
import atlite

import pandas as pd
import numpy as np
import geopandas as gpd
import xarray as xr
import networkx as nx

import panel as pn
import panel.widgets as pnw
import holoviews as hv

import cartopy.crs as ccrs

import hvplot.pandas
import hvplot.xarray
import hvplot.networkx as hvnx

In [ ]:
from bokeh.models.formatters import DatetimeTickFormatter
pn.extension()

In [ ]:
path = "../../pr/"

In [ ]:
import yaml
with open(path + "pypsa-eur-sec/config.yaml") as file:
    config = yaml.safe_load(file)

colors = config["plotting"]["tech_colors"]

## Solved Network

In [ ]:
n = pypsa.Network(
    path + "pypsa-eur-sec/results/your-run-name-overnight-dev/postnetworks/elec_s_60_lv1.25__Co2L0p0-365H-T-H-B-I-solar+p3-dist1_2030.nc"
)

## Geometry Polygon Data

In [ ]:
# shapes
nodes = gpd.read_file(path + "pypsa-eur/resources/regions_onshore_elec_s_60.geojson").set_index('name')
cts = gpd.read_file(path + "pypsa-eur/resources/country_shapes.geojson").set_index('name')

In [ ]:
regions = gpd.read_file(path + "pypsa-eur/resources/regions_onshore.geojson").append(
          gpd.read_file(path + "pypsa-eur/resources/regions_offshore.geojson"))
regions = regions.dissolve('name') 
onregions = gpd.read_file(path + "pypsa-eur/resources/regions_onshore.geojson").set_index('name')
regions["Area"] = regions.to_crs(epsg=3035).area.div(1e6)
onregions["Area"] = onregions.to_crs(epsg=3035).area.div(1e6)

## Model Inputs

In [ ]:
# country-level data
co2 = pd.read_csv(path + "pypsa-eur-sec/resources/co2_totals.csv", index_col=0)
energy = pd.read_csv(path + "pypsa-eur-sec/resources/energy_totals.csv", index_col=0)
transport = pd.read_csv(path + "pypsa-eur-sec/resources/transport_data.csv", index_col=0)
biomass = pd.read_csv(path + "pypsa-eur-sec/resources/biomass_potentials.csv", index_col=0)

In [ ]:
# nodal-level data
pop = pd.read_csv(path + "pypsa-eur-sec/resources/pop_layout_elec_s_60.csv", index_col=0)
idist = pd.read_csv(path + "pypsa-eur-sec/resources/industrial_distribution_key_elec_s_60.csv", index_col=0)
ienergy = pd.read_csv(path + "pypsa-eur-sec/resources/industrial_energy_demand_elec_s_60.csv", index_col=0)
iproduction = pd.read_csv(path + "pypsa-eur-sec/resources/industrial_production_elec_s_60.csv", index_col=0)
ienergy["total"] = ienergy.sum(axis=1)
iproduction["total"] = iproduction.sum(axis=1)

In [ ]:
def cmap(select):
    if "bio" in select:
        return "Greens"
    elif "solar" in select:
        return "Reds"
    elif "wind" in select:
        return "Blues"
    else:
        return "YlGnBu"

In [ ]:
def plot_geo(gdf, df, options, clim=None, tiles=None, alpha=1., line_width=0.6, select=True):

    if select:
        selector = pnw.Select(options=options)
    else:
        selector = pnw.RadioBoxGroup(options=options)

    def _plot(select):
        return gdf.hvplot(
            geo=True,
            height=700,
            c=df[select],
            tiles=tiles,
            alpha=alpha,
            line_width=line_width,
            cmap=cmap(select),
            clim=clim,
            hover_cols=['name']
        ).opts(
            active_tools=['pan', 'wheel_zoom']
        )

    plot = pn.bind(_plot, selector)
    widgets  = pn.Column(selector, plot)
    return widgets

### country-level

In [ ]:
plot_geo(cts, biomass, list(biomass.columns))

In [ ]:
plot_geo(cts, transport, list(transport.columns))

In [ ]:
plt_co2 = plot_geo(cts, co2, list(co2.columns))

In [ ]:
plt_energy = plot_geo(cts, energy, list(energy.columns))

## nodal level

In [ ]:
plot_geo(nodes, idist, list(idist.columns))

In [ ]:
plt_production = plot_geo(nodes, iproduction, list(iproduction.columns))

In [ ]:
plt_population = plot_geo(nodes, pop, ["total", "urban", 'rural'])

## Powerplantmatching

In [ ]:
plants = pd.read_csv("https://raw.githubusercontent.com/FRESNA/powerplantmatching/master/matched_data_red.csv", index_col=0)
plants = plants.loc[(plants.lat > 34) & (plants.lon < 72)]

In [ ]:
ppm_colors = {
    "Hydro": 'teal',
    "Hard Coal": 'black',
    "Lignite": 'grey',
    "Natural Gas": 'orange',
    "Nuclear": 'red',
    "Oil": 'brown',
    "Bioenergy": 'green',
    "Wind": '#235ebc',
    "Geothermal": 'purple',
    "Solar": '#f9d002',
    "Waste": "magenta",
    "Other": 'white',
}

In [ ]:
plt_powerplants = plants.hvplot.points(
    'lon',
    'lat',
    geo=True,
    frame_height=750,
    c='Fueltype',
    cmap=ppm_colors,
    size=plants["Capacity"] / 5,
    alpha=0.4,
    tiles='CartoLight',
    hover_cols=['Name', 'Fueltype', "Technology", 'YearCommissioned', "Retrofit", "Capacity"],
    xlim=(-12,32),
).opts(
    active_tools=['pan', 'wheel_zoom']
)

### Renewable Potentials Unclustered

In [ ]:
wind = pd.Series()
for profile in ['onwind', 'offwind-ac', 'offwind-dc']:
    ds = xr.open_dataset(f'{path}/pypsa-eur/resources/profile_{profile}.nc')
    wind = wind.append((ds.p_nom_max * ds.profile.sum('time')).to_pandas())
wind = wind.sum(level=0).reindex(regions.index, fill_value=0)
wind_per_skm = pd.DataFrame({"wind": wind / regions.Area / 1e3}) # GWh

In [ ]:
ds = xr.open_dataset(f'{path}/pypsa-eur/resources/profile_solar.nc')
solar = (ds.p_nom_max * ds.profile.sum('time')).to_pandas()

solar = solar.sum(level=0).reindex(onregions.index, fill_value=0)
solar_per_skm = pd.DataFrame({"solar": solar / onregions.Area / 1e3}) # GWh

In [ ]:
plt_wind_per_skm = plot_geo(regions, wind_per_skm, ["wind"], tiles='CartoLight', alpha=0.5, line_width=0.1)

In [ ]:
plt_solar_per_skm = plot_geo(onregions, solar_per_skm, ["solar"], tiles='CartoLight', alpha=0.7, line_width=0.1)

### Renewable Potentials Clustered

In [ ]:
cfs = n.generators_t.p_max_pu.groupby([n.generators.carrier, n.generators.bus.map(n.buses.location)], axis=1).mean().mean().unstack(0)

In [ ]:
plt_cfs = plot_geo(nodes, cfs, list(cfs.columns))

In [ ]:
pot = n.generators.p_nom_max.groupby([n.generators.carrier, n.generators.bus.map(n.buses.location)]).sum().unstack(0)

In [ ]:
pot.drop(index="EU", columns=['gas', 'oil', 'ror'], inplace=True)

In [ ]:
plt_pot = plot_geo(nodes, pot, list(pot.columns))

### Nodal Capacities and Costs

In [ ]:
term_p = "p_nom_opt"
term_e = "e_nom_opt"

In [ ]:
gen = n.generators.eval(term_p).groupby([n.generators.carrier, n.generators.bus.map(n.buses.location)]).sum()
sto = n.stores.eval(term_e).groupby([n.stores.carrier, n.stores.bus.map(n.buses.location)]).sum()
local_links = n.links.loc[n.links.bus0.map(n.buses.location) == n.links.bus1.map(n.buses.location)]
link = local_links.eval(term_p).groupby([local_links.carrier, local_links.bus0.map(n.buses.location)]).sum()
su = n.storage_units.eval(term_p).groupby([n.storage_units.carrier, n.storage_units.bus]).sum()

In [ ]:
gen = gen.unstack().drop("EU", axis=1).dropna(how='all')
link = link.unstack().drop("EU", axis=1).dropna(how='all')
sto = sto.unstack().drop("EU", axis=1).dropna(how='all')
su = su.unstack()

In [ ]:
cap = pd.concat([gen, sto, link, su]).T.div(1e3)

In [ ]:
cap[cap <= 0.1] = 0.

In [ ]:
plt_cap = plot_geo(nodes, cap, list(cap.columns))

## Base Networks

In [ ]:
base = pypsa.Network(path + "pypsa-eur/networks/base.nc")

In [ ]:
edge_ln_attrs = ["s_nom", "s_nom_opt", "v_nom", "type", "s_nom_extendable", "capital_cost", "under_construction", "underground"]
edge_lk_attrs = ["p_nom", "p_nom_opt", "type", "p_nom_extendable", "capital_cost", "under_construction", "underground", "underwater_fraction", "tags"]

In [ ]:
G_lines = nx.from_pandas_edgelist(base.lines.loc[base.lines.v_nom==380], 'bus0', 'bus1', edge_attr=edge_ln_attrs)
G_links = nx.from_pandas_edgelist(base.links.loc[base.links.carrier=='DC'], 'bus0', 'bus1', edge_attr=edge_lk_attrs)
pos = base.buses.loc[base.buses.carrier=='AC', ["x", "y"]].apply(tuple, axis=1).to_dict()

In [ ]:
network_map = cts.hvplot(
    geo=True,
    alpha=0.,
    height=850,
    width=1400,
    tiles="CartoLight",
) * \
hvnx.draw(
    G_links,
    pos=pos,
    responsive=True,
    geo=True,
    node_size=0,
    edge_color='royalblue',
    inspection_policy="edges",
    edge_width=2,
) * \
hvnx.draw(
    G_lines,
    pos=pos,
    geo=True,
    node_size=0,
    edge_color='firebrick',
    node_color='black',
    inspection_policy="edges",
    edge_width=2,
).opts(
    active_tools=['pan', 'wheel_zoom']
)

In [ ]:
if len(base.lines.v_nom.unique()) > 1:
    G_lines_300 = nx.from_pandas_edgelist(base.lines.loc[base.lines.v_nom==300], 'bus0', 'bus1', edge_attr=edge_ln_attrs)
    G_lines_220 = nx.from_pandas_edgelist(base.lines.loc[base.lines.v_nom==220], 'bus0', 'bus1', edge_attr=edge_ln_attrs)
    network_map *= \
    hvnx.draw(
        G_lines_300,
        pos=pos,
        geo=True,
        node_size=0,
        edge_color='orange',
        edge_width=1.5,
        inspection_policy="edges"
    ) * \
    hvnx.draw(
        G_lines_220,
        pos=pos,
        geo=True,
        node_size=0,
        edge_width=1,
        edge_color='green',
        inspection_policy="edges",
    )

## Networks

In [ ]:
G_lines = nx.from_pandas_edgelist(n.lines, 'bus0', 'bus1', edge_attr='s_nom_opt')

In [ ]:
G_links = nx.from_pandas_edgelist(n.links.loc[n.links.carrier=='DC'], 'bus0', 'bus1', edge_attr='p_nom_opt')

In [ ]:
H2 = n.links.loc[n.links.carrier=='H2 pipeline']
H2["location0"] = H2.bus0.apply(lambda x: x[:-3])
H2["location1"] = H2.bus1.apply(lambda x: x[:-3])
G_H2 = nx.from_pandas_edgelist(H2, 'location0', 'location1', edge_attr='p_nom_opt')
electrolysis = n.links.loc[n.links.carrier=='H2 Electrolysis'].groupby("bus0").p_nom_opt.sum()
nx.set_node_attributes(G_H2, electrolysis, "electrolysis")

In [ ]:
pos = n.buses.loc[n.buses.carrier=='AC', ["x", "y"]].apply(tuple, axis=1).to_dict()

In [ ]:
elec_net = nodes.hvplot(
    geo=True,
    color='whitesmoke',
    line_color='grey',
    line_width=0.5,
    #transform=ccrs.EuroPP(),
) * \
hvnx.draw(
    G_links, 
    pos=pos,
    width=1000,
    height=800,
    node_size=0,
    edge_color='navy',
    edge_width=hv.dim('p_nom_opt') / 3e3,
    geo=True,
    #crs=ccrs.EuroPP(),
    inspection_policy='edges'
) * \
hvnx.draw(
    G_lines, 
    pos=pos,
    width=1000,
    height=800,
    node_size=40,
    node_color='gray',
    edge_color='firebrick',
    edge_width=hv.dim('s_nom_opt') / 3e3,
    geo=True,
    #crs=ccrs.EuroPP(),
    inspection_policy='edges'
).opts(
    active_tools=['pan', 'wheel_zoom']
)

In [ ]:
h2_net = nodes.hvplot(
    geo=True,
    color='whitesmoke',
    line_color='grey',
    line_width=0.5,
) * \
hvnx.draw(
    G_H2, 
    pos=pos,
    width=1000,
    height=800,
    edge_color='cyan',
    edge_width=hv.dim('p_nom_opt') / 3e3,
    node_color='magenta',
    node_size=hv.dim("electrolysis") / 2e2,
    geo=True,
    inspection_policy='edges'
).opts(
    active_tools=['pan', 'wheel_zoom']
)

## System time series

In [ ]:
# one resampled version, one hourly version

In [ ]:
load = n.loads_t.p_set.groupby(n.loads.carrier, axis=1).sum()
formatter = DatetimeTickFormatter(months='%b')

In [ ]:
plt_load_ts = load.hvplot.area(width=1000, stacked=True, xformatter=formatter)

In [ ]:
selection = ["offwind-ac", "offwind-dc", "onwind", "ror", "solar"]
cfs = n.generators_t.p_max_pu.groupby(n.generators.carrier, axis=1).mean()[selection]

In [ ]:
cfs.hvplot.line(width=1000, xformatter=formatter)

In [ ]:
gen = n.generators_t.p.groupby(n.generators.carrier, axis=1).sum()

In [ ]:
su = n.storage_units_t.p.groupby(n.storage_units.carrier, axis=1).sum()

In [ ]:
df = pd.concat([gen, su], axis=1)

In [ ]:
plt_gen_ts = df.hvplot.area(width=1000, height=500, line_width=0, title="electricity generation")

# Cutouts

In [ ]:
era5 = atlite.Cutout(path + "pypsa-eur/cutouts/europe-2013-era5.nc")
sarah = atlite.Cutout(path + "pypsa-eur/cutouts/europe-2013-sarah.nc")

In [ ]:
def plot_cutout(cutout, variable):

    return cutout.data.hvplot.quadmesh(
        'x', 'y', variable,
        frame_height=700,
        cmap='Reds',
        coastline=True,
        project=True,
        geo=True,
        rasterize=True,
        ylim=(34,72),
        xlim=(-12,34),
        #clim=(0,1200),
        widget_location='top',
        #widgets={'time': pnw.DatetimeInput(value=dt.datetime(2013, 2, 8, 0, 0))}
        #tiles='CartoLight'
        #datashade=True, # removes legend
    )

In [ ]:
plt_cutout_wind = plot_cutout(era5, "wnd100m")

In [ ]:
plt_cutout_solar = plot_cutout(sarah, "influx_direct")

## Outputs

In [ ]:
import sys, os
sys.path.insert(0, os.getcwd() + "/" + path + "pypsa-eur-sec/scripts")
from plot_summary import rename_techs, preferred_order

In [ ]:
cost_df = pd.read_csv(
    path + "pypsa-eur-sec/results/your-run-name-overnight-dev/csvs/costs.csv",
    index_col=list(range(3)),
    header=list(range(4))
)
df = cost_df.groupby(cost_df.index.get_level_values(2)).sum()
df = df / 1e9
df = df.groupby(df.index.map(rename_techs)).sum()

to_drop = df.index[df.max(axis=1) < 1.]
                   
new_index = preferred_order.intersection(df.index).append(df.index.difference(preferred_order))
new_columns = df.sum().sort_values().index

In [ ]:
df.columns = [', '.join(col).strip() for col in df.columns.values]

In [ ]:
plt_scen_costs = df.T.hvplot.bar(
    stacked=True,
    rot=65,
    frame_width=800,
    frame_height=600,
    ylim=(0,1000),
    color='Category', cmap=colors)

In [ ]:
plt_scen_costs

In [ ]:
energy_df = pd.read_csv(
    path + "pypsa-eur-sec/results/your-run-name-overnight-dev/csvs/energy.csv",
    index_col=list(range(2)),
    header=list(range(4))
)
df = energy_df.groupby(energy_df.index.get_level_values(1)).sum()
df = df / 1e6
df = df.groupby(df.index.map(rename_techs)).sum()
to_drop = df.index[df.abs().max(axis=1) < 50]
df = df.drop(to_drop)
new_index = preferred_order.intersection(df.index).append(df.index.difference(preferred_order))
new_columns = df.columns.sort_values()

In [ ]:
df.columns = [', '.join(col).strip() for col in df.columns.values]

In [ ]:
plt_scen_energy = df.T.hvplot.bar(stacked=True, rot=65, frame_width=800, frame_height=600, ylim=(-20000,20000), color='Category', cmap=colors)

## Sankey
as https://holoviews.org/gallery/demos/bokeh/energy_sankey.html

In [ ]:
edges = pd.read_csv('../data/connection.csv')
sankey = hv.Sankey(edges, label='Energy Diagram')
sankey.opts(label_position='left', edge_color='target', node_color='index', cmap=colors)

## Industry Sector Ratios

In [ ]:
iratios = pd.read_csv(path + "pypsa-eur-sec/resources/industry_sector_ratios.csv", index_col=0)

In [ ]:
plt_iratios = iratios.T.hvplot.barh(stacked=True, width=1000, height=400, title="Industry Sector Ratios [MWh/t material]")

## Hotmaps Raw

In [ ]:
def prepare_hotmaps_database(regions):
    """
    Load hotmaps database of industrial sites and map onto bus regions.
    """

    df = pd.read_csv(path + "pypsa-eur-sec/data/Industrial_Database.csv", sep=";", index_col=0)

    df[["srid", "coordinates"]] = df.geom.str.split(';', expand=True)

    # remove those sites without valid locations
    df.drop(df.index[df.coordinates.isna()], inplace=True)

    df['coordinates'] = gpd.GeoSeries.from_wkt(df['coordinates'])

    gdf = gpd.GeoDataFrame(df, geometry='coordinates', crs="EPSG:4326")

    gdf = gpd.sjoin(gdf, regions, how="inner", op='within')

    gdf.rename(columns={"index_right": "bus"}, inplace=True)
    gdf["country"] = gdf.bus.str[:2]

    return gdf

In [ ]:
hotmaps = prepare_hotmaps_database(nodes)

hotmaps["geometry"] = hotmaps.coordinates
hotmaps["lat"] = hotmaps.geometry.y
hotmaps["lon"] = hotmaps.geometry.x

In [ ]:
plt_hotmaps = hotmaps.hvplot.points(
    'lon',
    'lat',
    geo=True,
    frame_height=750,
    c='Subsector',
    size=hotmaps["Emissions_ETS_2014"] / 2e3,
    alpha=0.4,
    tiles='CartoLight',
    hover_cols=['SiteName', "Emissions_ETS_2014", "DataSource"],
).opts(
    active_tools=['pan', 'wheel_zoom']
)

In [ ]:
import panel as pn
import numpy as np
import holoviews as hv

pn.extension()

board = pn.template.BootstrapTemplate(title='PyPSA-Eur-Sec Dashboard', header_background="#d95568")

pn.config.sizing_mode = 'stretch_width'

intro = pn.pane.Markdown('''

PyPSA-Eur-Sec is an open model dataset of the European energy system
at the transmission network level that covers the full ENTSO-E area.

[pypsa-eur-sec.readthedocs.io](https://pypsa-eur-sec.readthedocs.io/)
''')

_network_map = pn.Row(
    network_map
)

_existing = pn.Row(
    pn.Card(plt_hotmaps, collapsible=False, title="Industrial Sites"),
    pn.Card(plt_powerplants, collapsible=False, title="Powerplants")
)

_potentials = pn.Row(
    pn.Card(plt_wind_per_skm, collapsible=False, title="Wind Potential"),
    pn.Card(plt_solar_per_skm, collapsible=False, title="Solar Potential"),
)

_cutouts = pn.Row(
    pn.Card(plt_cutout_wind, collapsible=False, title="Wind Speeds"),
    pn.Card(plt_cutout_solar, collapsible=False, title="Direct Influx")
)

_clustered_potentials = pn.Row(
    pn.Card(plt_cfs, collapsible=False, title="Capacity Factors"),
    pn.Card(plt_pot, collapsible=False, title="Potential")
)

_totals = pn.Row(
    pn.Card(plt_energy, collapsible=False, title="Energy Consumption"),
    pn.Card(plt_co2, collapsible=False, title="Carbon Emissions")
)

_opt_nets = pn.Row(
    pn.Card(elec_net, collapsible=False, title="Electricty Network"),
    pn.Card(h2_net, collapsible=False, title="Hydrogen Network")
)

_timeseries = pn.Column(
    pn.Card(plt_load_ts, collapsible=False, title="Load"),
    pn.Card(plt_gen_ts, collapsible=False, title="Electricity Generation")
)

board.sidebar.append(intro)

board.main.append(pn.pane.Markdown(""" """))

board.main.append(
    pn.Tabs(
        ("Transmission Network", _network_map),
        ("Existing Infrastructure", _existing),
        ("Renewable Potentials", _potentials),
        ("Cutouts", _cutouts),
        ("Renewable Potentials (clustered)", _clustered_potentials),
        ("Energy and CO2", _totals),
        ("Industry Sector Ratios", plt_iratios),
        ("Energy", plt_scen_energy),
        ("System Costs", plt_scen_costs),
        ("Optimised Networks", _opt_nets),
        ("Optimised Capacities", plt_cap),
        ("Timeseries", _timeseries),
        dynamic=True
    )
)

board.show()